# CSMODEL Project - Netflix Userbase Dataset Case Study

### Group 8
CAPAROS, MIGUEL ANTONIO <br> 
FERRER, ANGEL JUNE <br>
MARTINEZ, AZELIAH <br>
VILLANUEVA, KEISHA LEIGH <br>

# I. Dataset Description

The Netflix Userbase Dataset provides a snapshot of a sample Netflix userbase, showcasing various aspects of user subscriptions, revenue, account details, and activity. Each row represents a unique user, identified by their User ID. The dataset serves as a synthetic representation and does not reflect actual Netflix user data. 


## Data Collection Process

The dataset is synthetically sourced, and as such, any conclusions and insights may not accurately reflect real-world data. 



## Dataset File Structure

Each row in the dataset represents a unique user, identified by their User ID. The columns contain various details about the user, including:

- User ID: A unique identifier for each user.
- Subscription Type: The type of subscription the user has (basic, standard, or premium).
- Monthly Revenue: The revenue generated from the user's subscription each month.
- Join Date: The date the user joined Netflix.
- Last Payment Date: The date of the user's most recent payment.
- Country: The country where the user is located.
- Age: The user's age.
- Gender: The user's gender.
- Device Type: The type of device used to access Netflix (e.g., Smart TV, Mobile, Desktop, Tablet).
- Plan Duration: The duration of the user's current subscription plan.

The dataset contains a total of 2500 observations (rows) and 10 variables (columns). Each variable provides specific details about the users, enabling analysis of subscription patterns, revenue generation, and user behavior.

**If the dataset is composed of different files that you will combine in the succeeding steps, describe the structure and the contents of each file.**


## Dataset Variables

What does each variable represent? All variables, even those which are not used for the study, should be described to the reader. The purpose of each variable in the dataset should be clear to the reader of the notebook without having to go through an external link.

# II. Data Cleaning

?



# III. Exploratory Data Analysis

In this section of the notebook, you must fulfill the following:
- Identify at least __4 exploratory data analysis questions__. Properly state the questions in the notebook. Having more than 4 questions is acceptable, especially if this will help in understanding the data better.

Answer the EDA questions using both:
- Numerical Summaries – measures of central tendency, measures of dispersion, and correlation
- Visualization – Appropriate visualization should be used. Each visualization should be accompanied by a brief explanation.

To emphasize, __both numerical summary and visualization__ should be presented for each question.
The whole process should be supported with verbose textual descriptions of your procedures and findings.
